# NNAST Model Training on Google Colab

This notebook trains the NNAST model using GPU acceleration.


## 1. 環境セットアップ


In [ ]:
# GPU確認
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")


In [ ]:
# リポジトリのクローンまたはコードのアップロード
import os
import zipfile

# 方法1: Gitリポジトリからクローン（推奨）
# !git clone https://github.com/yourusername/nnast_model.git
# os.chdir('nnast_model')

# 方法2: zipファイルをアップロードして解凍
# from google.colab import files
# uploaded = files.upload()
# for filename in uploaded.keys():
#     if filename.endswith('.zip'):
#         with zipfile.ZipFile(filename, 'r') as zip_ref:
#             zip_ref.extractall('.')
#         print(f"Extracted {filename}")


In [ ]:
# 依存関係のインストール
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install torch-geometric
!pip install transformers
!pip install tqdm


## 2. データセットのアップロード


In [ ]:
# 方法1: Google Driveからマウント（推奨）
from google.colab import drive
drive.mount('/content/drive')

# データセットをGoogle Driveにアップロードしてから、ここでパスを指定
# 例: /content/drive/MyDrive/nnast_dataset/training_data
DATASET_BASE = "/content/drive/MyDrive/nnast_dataset"


In [ ]:
# 方法2: 直接アップロード（小規模データセット用）
# from google.colab import files
# uploaded = files.upload()
# 
# import zipfile
# for filename in uploaded.keys():
#     if filename.endswith('.zip'):
#         with zipfile.ZipFile(filename, 'r') as zip_ref:
#             zip_ref.extractall('.')
#         print(f"Extracted {filename}")


## 3. 学習の実行


In [ ]:
# 学習パラメータ
GRAPHS_FILE = f"{DATASET_BASE}/training_data/train_graphs.jsonl"  # パスを調整
LABELS_FILE = f"{DATASET_BASE}/training_data/train_labels.jsonl"
OUTPUT_DIR = "./checkpoints"

# GPU用の最適化設定
BATCH_SIZE = 16  # GPUなら大きく設定可能（メモリに応じて調整）
EPOCHS = 10
MAX_NODES = 1000
HIDDEN_DIM = 256
LEARNING_RATE = 1e-4


In [ ]:
# 学習実行
!python3 -m ml.train \
    --graphs {GRAPHS_FILE} \
    --labels {LABELS_FILE} \
    --output-dir {OUTPUT_DIR} \
    --batch-size {BATCH_SIZE} \
    --epochs {EPOCHS} \
    --hidden-dim {HIDDEN_DIM} \
    --max-nodes {MAX_NODES} \
    --device cuda \
    --gnn-type GAT \
    --lr {LEARNING_RATE}


## 4. 結果の確認とダウンロード


In [ ]:
# 学習履歴の可視化
import json
import matplotlib.pyplot as plt

history_file = f"{OUTPUT_DIR}/training_history.json"
if os.path.exists(history_file):
    with open(history_file) as f:
        history = json.load(f)
    
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    
    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Train Acc')
    plt.plot(history['val_acc'], label='Val Acc')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.title('Training and Validation Accuracy')
    
    plt.tight_layout()
    plt.show()
else:
    print("Training history file not found")


In [ ]:
# チェックポイントをGoogle Driveに保存
import shutil

drive_checkpoint_dir = f"{DATASET_BASE}/checkpoints"
os.makedirs(drive_checkpoint_dir, exist_ok=True)

if os.path.exists(OUTPUT_DIR):
    # 既存のディレクトリをコピー
    if os.path.exists(drive_checkpoint_dir):
        shutil.rmtree(drive_checkpoint_dir)
    shutil.copytree(OUTPUT_DIR, drive_checkpoint_dir)
    print(f"✓ Checkpoints saved to {drive_checkpoint_dir}")
else:
    print("Checkpoint directory not found")
